In [1]:
import dynamic_traffic_signal
import importlib

In [6]:
importlib.reload(dynamic_traffic_signal)

<module 'dynamic_traffic_signal' from '/home/utkarsh/Study/Quantela/Traffic_Simulation/dynamic_traffic_signal.py'>

In [2]:
a = dynamic_traffic_signal.DynamicTrafficSignal('Testing_class_ijipura')

In [3]:
a.optimize_traffic_lights(timing_range={'min':18, 'max':21}, gui=True)

18 18
 Retrying in 1 seconds
18 19
 Retrying in 1 seconds
18 20
 Retrying in 1 seconds
19 18
 Retrying in 1 seconds
19 19
 Retrying in 1 seconds
19 20
 Retrying in 1 seconds
20 18
 Retrying in 1 seconds
20 19
 Retrying in 1 seconds
20 20
 Retrying in 1 seconds
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:48415 [Errno 111] Connection refused
 Retrying in 2 seconds


{'one': 18, 'two': 20}

In [4]:
a.optimized_result

{'one': 18, 'two': 20}

In [57]:
import pandas as pd
import numpy as np
import networkx as nx
import xml.etree.ElementTree as Xml
from geopy.distance import great_circle


class OsmNetworkInfo:
    """
    Here we are parsing and extracting relevenat information from osm file to use in our model
    """

    def __init__(self, xml_name_location):
        self.xml_name_location = xml_name_location

    def get_raw_traffic_light_nodes(self):
        """
        here we are finding all raw traffic light nodes
        :return: dictionary of all traffic light nodes
        """
        e = Xml.parse(self.xml_name_location).getroot()
        raw_traffic_light_nodes = {}
        for i in e:
            if i.tag == "node":
                junction_name = "None"
                isSignal = False
                for j in i:
                    
                    if j.tag == "tag":
                        if j.attrib["k"] in ["designation", "name"]:
                            junction_name = j.attrib["v"]
                            
                        if j.attrib["v"] == "traffic_signals":
                            isSignal = True
                            raw_traffic_light_nodes[i.attrib["id"]] = {'lat': i.attrib["lat"],
                                                                       'long': i.attrib["lon"]}
                            
                if isSignal:
                    raw_traffic_light_nodes[i.attrib["id"]]['junction_name'] = junction_name
                        
        return raw_traffic_light_nodes

    def get_nodes_to_merge(self):
        """
        here we are finding the nodes of junction which has to be merged is sumo to get proper junction
        :return: list of lists that contain nodes to be merged
        """
        graph = nx.Graph()
        e = Xml.parse(self.xml_name_location).getroot()
        raw_nodes = self.get_raw_traffic_light_nodes()
        for i in e:
            if i.tag == "way":
                way_tmp = []
                insert = True
                for j in i:
                    if j.tag == "nd":
                        if j.attrib["ref"] in raw_nodes.keys():
                            way_tmp.append(j.attrib["ref"])
                        else:
                            insert = False
                            break
                if insert:
                    graph.add_path(way_tmp)
        nx.set_node_attributes(graph, raw_nodes)
        
        for i in graph.edges:
            distance = great_circle((graph.nodes[i[0]]['lat'], graph.nodes[i[0]]['long']),
                                    (graph.nodes[i[1]]['lat'], graph.nodes[i[1]]['long'])).meters
            if distance > max_distance:
                graph.remove_edge(*i)

        output = {}
        i = 0
        print(nx.connected_component_subgraphs(graph))
        for h in nx.connected_component_subgraphs(graph):
            output[i] = {'nodes': list(h.nodes), 'suggested_name':'None'}
            print(h.nodes)
            for node in h.nodes:
                if h.nodes[node]['junction_name'] is not 'None':
                    output[i]['suggested_name'] = h.nodes[node]['junction_name']
            i += 1

        return output



In [58]:
a = OsmNetworkInfo('test.osm')

In [59]:
net = a.get_nodes_to_merge()

<generator object connected_component_subgraphs at 0x7f619310a048>
['315208341', '315208342']


In [60]:
net

{0: {'nodes': ['315208341', '315208342'], 'suggested_name': 'Ejipur Signal'}}

In [28]:
from geopy.distance import great_circle
max_distance = 20
for i in net.edges:
    distance = great_circle((net.nodes[i[0]]['lat'], net.nodes[i[0]]['long']),
                           (net.nodes[i[1]]['lat'], net.nodes[i[1]]['long'])).meters
    if distance > max_distance:
        net.remove_edge(*i) 

In [29]:
a.get_raw_traffic_light_nodes()

{'61748555': {'lat': '12.9516788',
  'long': '77.6214774',
  'junction_name': 'None'},
 '260729391': {'lat': '12.9372613',
  'long': '77.6269775',
  'junction_name': 'Sony World Signal'},
 '263227076': {'lat': '12.9298524',
  'long': '77.6331784',
  'junction_name': 'None'},
 '309592695': {'lat': '12.9351857',
  'long': '77.6244386',
  'junction_name': 'None'},
 '315208341': {'lat': '12.9385881',
  'long': '77.6326767',
  'junction_name': 'None'},
 '315208342': {'lat': '12.9384851',
  'long': '77.6327639',
  'junction_name': 'Ejipur Signal'}}

In [7]:
net.edges

EdgeView([('315208341', '315208342')])